In [1]:
import csv
import numpy as np
import pandas as pd


In [2]:
data = pd.read_csv(r"answer_sql12.csv", sep='|')
len(data)

328688

In [3]:
relax_rate = 1.05
contract_rate = 0.95
relax_rate_change = [1.1, 1.2, 1.3, 1.4, 1.5, 1.6]
contract_rate_change = [0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

# relax

In [4]:
data1 = data[(data['o_orderpriority'] == "5-LOW") & (data["l_linenumber"] == 3) ]
print(len(data1))
print(len(data1)*relax_rate)

11757
12344.85


In [5]:
data2 = data[(data['l_shipinstruct'] == "COLLECT COD")]
print(len(data2))
print(len(data2)*relax_rate)

82202
86312.1


In [6]:
for r in relax_rate_change:
    print(len(data2) * r)

90422.20000000001
98642.4
106862.6
115082.79999999999
123303.0
131523.2


In [7]:
data3 = data[data["l_returnflag"] == "A"]
print(len(data3))
print(len(data3)*relax_rate)

164535
172761.75


## contract

In [8]:
data3 = data[(data['o_orderpriority'] == "1-URGENT")]
print(len(data3))
print(len(data3)*contract_rate)

65861
62567.95


In [9]:
data4 = data[(data['l_linenumber'] == 1)]
print(len(data4))
print(len(data4)*contract_rate)

82008
77907.59999999999


In [10]:
data1 = data[(data['l_returnflag'] == "A") & (data["o_orderstatus"] == "F") ]
print(len(data1))
print(len(data1)*contract_rate)

164535
156308.25


In [11]:
for r in contract_rate_change:
    print(len(data1) * r)

65814.0
82267.5
98721.0
115174.49999999999
131628.0
148081.5


# refine

In [12]:
data3 = data[(data["o_orderpriority"] == "3-MEDIUM") & (data["l_linenumber"] == 3) ]
print(len(data3))
print(len(data3)*1.05)

11692
12276.6


In [13]:
data4 = data[(data['l_returnflag'] == "R") & (data["l_shipinstruct"] == "TAKE BACK RETURN") ]
print(len(data4))
print(len(data4)*0.95)

40821
38779.95


In [14]:
s = [0.88, 0.90, 0.92, 0.94, 0.96, 0.98]
for r in s:
    print(len(data4) * r)

35922.48
36738.9
37555.32
38371.74
39188.159999999996
40004.58


In [15]:
data['l_returnflag'].unique()

array(['R', 'A'], dtype=object)

In [16]:
s = data['l_receiptdate'].unique().tolist()
s.sort()
print(s[0], s[-1])

1994-01-01 1995-01-30


# refine5

In [4]:


g1 = data[(data["o_orderpriority"] == "4-NOT SPECIFIED") & (data["l_linenumber"] == 7) & (data["l_shipinstruct"] == "TAKE BACK RETURN") & (data["l_returnflag"] == "A")]

print(len(g1))
print(len(g1) * 0.95)



306
290.7


In [5]:

g2 = data[(data["l_shipinstruct"] == "NONE") & (data["o_orderpriority"] == "5-LOW")& (data["l_linenumber"] == 7) & (data["o_orderstatus"] == "F")]

print(len(g2))
print(len(g2) * 1.05)

609
639.45


In [6]:


s = [1.02, 1.04, 1.06, 1.08, 1.10, 1.12]
for r in s:
    print(len(g2) * r)

621.1800000000001
633.36
645.5400000000001
657.72
669.9000000000001
682.08


# join

In [17]:

orders = pd.read_csv("../../../../../InputData/TPC-H/10Gdata/orders.tbl", sep='|')
lineitem = pd.read_csv("../../../../../InputData/TPC-H/10Gdata/lineitem.tbl", sep='|')
joined = pd.merge(left=orders, right=lineitem, how="inner", left_on="o_orderkey", right_on="l_orderkey")
joined = joined[joined["l_commitdate"] < joined["l_receiptdate"]]
joined = joined[joined["l_shipdate"] < joined["l_commitdate"]]

In [18]:
print(len(joined))

7190268


In [22]:
joined.to_csv("sql12_joined.csv", sep='|', index=False)

In [23]:
# data analysis

In [24]:
joined = pd.read_csv("sql12_joined.csv", sep='|')

# original query 1

In [27]:
# original query 2
data1 = joined[(joined['l_receiptdate'] >=  19940101)]
data1 = data1[(data1['l_shipdate'] <  19950101)]
data1 = data1[(data1['l_shipmode'] == "MAIL") | (data1['l_shipmode'] == "FOB")]
print(len(data1))

g1 = data1[(data1["o_orderpriority"] == "3-MEDIUM") & (data1["l_linenumber"] == 7) & (data1["l_shipinstruct"] == "TAKE BACK RETURN")]

print(len(g1))
print(len(g1) * 0.95)


g2 = data1[(data1["l_shipinstruct"] == "NONE") & (data1["o_orderpriority"] == "5-LOW")& (data1["l_linenumber"] == 7)]

print(len(g2))
print(len(g2) * 1.05)


328688
601
570.9499999999999
609
639.45
